In [ ]:
#1 install the package

package_path = " .."
requirements = "../requirements.txt"

!python3 -m pip install -r $requirements
!python3 -m pip install -e $package_path

## Sample AI Config Representation
```json
{
    "name": "gpt4 as your data engineer",
    "description": "",
    "schema_version": "latest",
    "metadata": {
        "parameters": {},
        "models": {
            "gpt-3.5-turbo": {
                "model": "gpt-3.5-turbo",
                "top_p": 1,
                "max_tokens": 3000,
                "temperature": 1
            },
            "gpt-4": {
                "model": "gpt-4",
                "top_p": 1,
                "max_tokens": 3000,
                "temperature": 1,
                "system_prompt": "You are an expert at SQL. You will output nicely formatted SQL code with labels on columns. You will provide a short 1-2 sentence summary on the code. Name columns as one word using underscore and lowercase. Format Output in markdown ### SQL Query code block with SQL Query &nbsp; ### Summary short summary on code"
            }
        }
    },
    "prompts": [
        {
            "name": "prompt1",
            "input": "Write me a {{sql_language}} query to get this final output: {{output_data}}. Use the tables relationships defined here: {{table_relationships}}.",
            "metadata": {
                "model": {
                    "name": "gpt-3.5-turbo"
                }
            }
        },
        {
            "name": "prompt2",
            "input": "Translate the following into PostgreSQL code:\n {{prompt1.output}}",
            "metadata": {
                "model": {
                    "name": "gpt-4",
                    "settings": {
                        "model": "gpt-4",
                        "top_p": 1,
                        "max_tokens": 3000,
                        "temperature": 1
                        
                    }
                }
            }
        }
    ]
}```

In [ ]:
# Load A Config

from aiconfig_tools import AIConfigRuntime

config_file_path = "parametrized_data_config.json"
config = AIConfigRuntime.from_config(config_file_path)

In [ ]:
# Prompt1 takes in 3 parameters: sql_language, output_data, table_relationships
# Lets define some parameter values for prompt1

prompt1_params = {
    "sql_language": "mysql",
    "output_data": "user_name, user_email, trial, num_trial_steps, num_trial_steps_params. output granularity is the trial_id.",
    "table_relationships": "user table, trial table, trial_step table. a user can create many trials. each trial has many trial_steps. a trial_step has parameters if metadata[0] (json) has a non-null parameters value. "
}

In [ ]:
# What if I want to see the completion parameters of a prompt before inference? This is useful for debugging, calculating cost of api calls, etc.

%env OPENAI_API_KEY=0
prompt1_completion_params = await config.resolve("prompt1", prompt1_params) 

import pprint
# Lets take a look at the completion parameters
pprint.pprint(prompt1_completion_params)

In [ ]:
# Great! Now lets run prompt1 with the above parameters. Make sure to have your api credentials set.
import asyncio

prompt1_output = await config.run(prompt1_params, "prompt1")
print(prompt1_output)

In [ ]:
# Streaming

from aiconfig_tools import InferenceOptions

# Lets define a handler for streaming OpenAI API calls. This function will be called for every delta in the output stream.
def print_stream_delta(data, accumulated_data, index: int):
    """
    streamCallback function that prints the stream output to console.
    """
    if "content" in data:
        content = data['content']
        print(content, end = "", flush=True)


# Define inference options to stream the output of the API call.
inference_options = InferenceOptions(stream_callback=print_stream_delta, stream=True)

# Run with streaming enabled, as well as the other parameters we defined earlier.
await config.run(prompt1_params, "prompt1", inference_options) 


In [ ]:
# Chained Parameterized Prompts

# prompt 2 uses prompt1's output. Prompt2 asks to translate into postgres
await config.run({}, "prompt2", inference_options)

In [ ]:
# CRUD API
# Programatically add, delete, and update prompts in the config.
import json
from aiconfig_tools.AIConfigSettings import Prompt

# Lets define a new prompt
# Chat History defaults to True. Lets explicitly set it to false.
prompt = Prompt(**{"name": "prompt3", 
                   "input": "How do transformers work?", 
                   "metadata": {
                       "model": "gpt-3.5-turbo",
                       "remember_chat_context": False,
                }})

config.delete_prompt("prompt3")
config.add_prompt("prompt3", prompt)

# Exports aiconfig to disk.
config.save("updated_aiconfig.json")

# Let's check out the new update config

with open('updated_aiconfig.json', 'r') as f:
    parsed = json.load(f)
    pprint.pprint(parsed)


In [ ]:
config2 = AIConfigRuntime.from_config("updated_aiconfig.json")

print(await config2.resolve( "prompt3",{}, inference_options))